This function computes the gradient of the velocity-field $ \mathbf{v}(\mathbf{x}, t) $ using an auxiliary meshgrid.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| t | float | time |
| x | array (2,) | $ \mathbf{x} $ |
| X | array (NY, NX) | X-meshgrid of velocity data|
| Y | array (NY, NX) | Y-meshgrid of velocity data|
| Interpolant_u | Interpolant object | Interpolant object for $ u(\mathbf{x}, t)  $ |
| Interpolant_v | Interpolant object | Interpolant object for $ v(\mathbf{x}, t)  $ |
| periodic | list (3,) | periodic[0]: periodicity in x <br /> periodic[1]: periodicity in y <br /> periodic[2]: periodicity in time|
| defined_domain | array (NY, NX) | points on the meshgrid where velocity field is defined |
| bool_unsteady | bool | specifies if velocity field is unsteady/steady |
| time_data | array(1,NT) | time of velocity data |
| delta_data | list (2,) | delta[0]: x-grid spacing of the velocity data <br /> delta[1]: y-grid spacing of the velocity data|
| aux_grid | list (2,) | aux_grid[0]: dx_auxiliary spacing <br /> aux_grid[1]: dy_auxiliary spacing |
| grad_vel | array (2,2) | $ \nabla \mathbf{v}(\mathbf{x}, t) $ |

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-1])

# add integration folder to current working path
sys.path.append(parent_directory+"/integration")

In [2]:
# Import numpy as np
import numpy as np

# Import function to compute velocity
from ipynb.fs.defs.velocity import velocity

# Import function to check particle location
from ipynb.fs.defs.check_location import check_location

In [3]:
def gradient_velocity(t, x, X, Y, Interpolant_u, Interpolant_v, periodic, defined_domain, bool_unsteady, time_data, delta_data, aux_grid):
    '''This function computes the gradient of the velocity-field by finite differencing on an auxiliary grid.

    Parameters:
        t: float, time
        x: array (2,Npoints),  vector of positions
        X: array (Ny, Nx), X-meshgrid
        Y: array (Ny, Nx), Y-meshgrid
        Interpolant_u: Interpolant object for u(x,t)
        Interpolant_v: Interpolant object for v(x,t)
        Interpolant_w: Interpolant object for w(x,t)
        periodic: list (2,), periodic[0]: periodicity in x
                             periodic[1]: periodicity in y
        bool_unsteady: bool, specifies if velocity field is unsteady/steady
        time_data: array(Nt, ), time spacing of the velocity data
        delta_data: list (2,), delta[0]: x-grid spacing of the velocity data
                          delta[1]: y-grid spacing of the velocity data
        aux_grid: float, ratio of auxiliary meshgrid to original meshgrid
        
    Returns:
        grad_vel: array (2,2, Npoints),  gradient (Jacobian matrix) of the velocity field
    '''    
    x = x.reshape(2,-1)
    
    # define auxiliary grid spacing
    rho_x = aux_grid[0] # float
    rho_y = aux_grid[1] # float
    
    X0, XL, XR, XU, XD = [], [], [], [], []
    
    for i in range(x.shape[1]):
        
        xr = x[0, i] + rho_x # float
        xl = x[0, i] - rho_x # float 
         
        if periodic[0]:
            xr = xr%(X[0,-1]-X[0,0])+X[0,0] # float
            xl = xl%(X[0,-1]-X[0,0])+X[0,0] # float
        
        yu = x[1, i] + rho_y # float
        yd = x[1, i] - rho_y # float
        
        if periodic[1]:
            yu = yu%(Y[-1,0]-Y[0,0])+Y[0,0] # float
            yd = yd%(Y[-1,0]-Y[0,0])+Y[0,0] # float
    
        X0.append([x[0, i], x[1, i]])
        XL.append([xl, x[1, i]])
        XR.append([xr, x[1, i]])
        XU.append([x[0, i], yu])
        XD.append([x[0, i], yd])    
    
    X0 = np.array(X0).transpose()
    XL = np.array(XL).transpose()
    XR = np.array(XR).transpose()
    XU = np.array(XU).transpose()
    XD = np.array(XD).transpose()
    
    # velocity on the auxiliary meshgrid
    vLend = velocity(t, XL, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data)    
    vRend = velocity(t, XR, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data)     
    vDend = velocity(t, XD, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data)   
    vUend = velocity(t, XU, X, Y, Interpolant_u, Interpolant_v, periodic, bool_unsteady, time_data)
    
    grad_vel = np.zeros((2, 2, vLend.shape[1]))
    
    # compute gradient of velocity
    
    for i in range(X0.shape[1]):
        
        grad_vel[0, 0, i] = (vRend[0,i]-vLend[0,i])/(2*rho_x)
        grad_vel[1, 0, i] = (vRend[1,i]-vLend[1,i])/(2*rho_x)
        grad_vel[0, 1, i] = (vUend[0,i]-vDend[0,i])/(2*rho_y)
        grad_vel[1, 1, i] = (vUend[1,i]-vDend[1,i])/(2*rho_y)
    
    return grad_vel